# CS475 Fall 2021 Homework 3

This notebook is based on the [Huggingface's official example](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb). Note that you should submit `bert_poolers.py`, not `.ipynb` file converted from this colab notebook. Please refer to the homework description for more details.

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers

/bin/bash: /home/dhkim/anaconda3/envs/rlgpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/text-classification).

**You have to implement the following block, which is the same as `bert_poolers.py`.**

In [2]:
import torch
import torch.nn as nn
from transformers import BertConfig
from transformers.models.bert.modeling_bert import (
    BertPreTrainedModel, BertModel,
    BertEmbeddings, BertEncoder, BertForSequenceClassification, BertPooler,
)


class MeanMaxTokensBertPooler(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.w_mmt = nn.Linear(2*config.hidden_size, config.hidden_size) # W_MMT
        self.activation = nn.Tanh()

        self.hidden_size = config.hidden_size
        # raise NotImplementedError

    def forward(self, hidden_states, *args, **kwargs):
        norm = hidden_states.norm(dim=2) # [N, T]
        max_id = torch.argmax(norm, dim=1) # [N]
        max_id = torch.reshape(max_id, (-1, 1, 1)) # [N, 1, 1]
        indices = torch.broadcast_to(max_id, (-1, 1, self.hidden_size)) # [N, 1, H]
        maxT = torch.gather(hidden_states, 1, indices) # [N, 1, H]

        # option 1: ( sum(T_i) ) || max(T_i)
        sumT = torch.mean(hidden_states, dim=1) # [N, H]
        c_mmt = torch.cat((sumT, maxT.squeeze()), dim=1) # [N, 2H]

        # # option 2: sum( (T_i || max(T_i)) )
        # maxT = torch.broadcast_to(maxT, hidden_states.shape) # [N, T, H]
        # concatT = torch.cat((hidden_states, maxT), dim=2) # [N, T, 2H]
        # c_mmt = torch.sum(concatT, dim=1) # [N, 2H]
        
        pooled_output = self.w_mmt(c_mmt) # [N, H]
        pooled_output = self.activation(pooled_output) # [N, H]

        return pooled_output
        # raise NotImplementedError


class MyBertPooler(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        # MyBertPooler
        # "RNN" / "Linear" / "Attentive" / "Min" / "Max" / "Mean"
        self.mode = "Attentive2"

        # 1-1. RNN-based
        self.w_mypooler = nn.LSTM(input_size = config.hidden_size, 
                                 hidden_size = config.hidden_size, 
                                 num_layers = 1,
                                 batch_first=True)
        
        # 1-2. Linear 
        # self.w_linear_2 = nn.Linear()
        # self.w_linear_1 = nn.Linear(config.hidden_size, 1)
        self.w_linear_1 = None

        # 1-3. Residual Linear 
        self.w_residual_1 = nn.Linear(config.hidden_size, config.hidden_size)
        self.w_residual_2 = nn.Linear(config.hidden_size, 1)
        self.w_residual_s = nn.Linear(config.hidden_size, 1)

        # 1-4. CNN

        # 2. Attentive Pooling
        # self.w_att_simple = nn.Linear()
        self.query = nn.Linear(config.hidden_size, 1)
        self.softmax = nn.Softmax()
        self.w_2h = nn.Linear(2*config.hidden_size, config.hidden_size) # w_2h

        # 3. Min / Max / Average
        self.w_basic = nn.Linear(config.hidden_size, config.hidden_size) # W_basic

        self.hidden_size = config.hidden_size
        self.activation = nn.Tanh()
        # raise NotImplementedError

    def forward(self, hidden_states, *args, **kwargs):
        # hidden_states: [N, T, H]
        
        if (self.mode == "RNN"): # 1-1. RNN-based
          output, _ = self.w_mypooler(hidden_states) # [N, T, D * H], _
          pooled_output = output[:, -1, :] # [N, H]
          # pooled_output = torch.mean(output, dim=1)
          pooled_output = self.activation(pooled_output) # [N, H]
          return pooled_output

        elif (self.mode == "Linear"): # 1-2. Linear
          # h_T = torch.transpose(hidden_states, 1, 2) # [N, H, T]
          # pooled_output = self.w_linear_1(h_T) # [N, H, 1]
          # pooled_output = self.activation(pooled_output.squeeze()) # [N, H]
          if self.w_linear_1 is None:
            self.w_linear_1 = nn.Linear(hidden_states.shape[1], 1).to('cuda')
          h_T = torch.transpose(hidden_states, 1, 2) # [N, H, T]
          pooled_output = self.w_linear_1(h_T)
          pooled_output = self.activation(pooled_output.squeeze()) # [N, H]
          # raise NotImplementedError
          return pooled_output

        elif (self.mode == "Residual"): # 1-3. Residual Linerar
          h_T = torch.transpose(hidden_states, 1, 2) # [N, H, T]
          pooled_output = self.w_residual_1(h_T) # [N, H, T]
          pooled_output = self.w_residual_2(pooled_output) # [N, H, 1]
          pooled_output = self.activation(pooled_output.squeeze()) # [N, H]
          pooled_output = pooled_output + (self.w_residual_s(h_T)).squeeze()
          return pooled_output

        elif (self.mode == "Attentive"): # 2. Attentive Pooling
          score = self.query(hidden_states)
          score /= torch.sqrt(torch.tensor(self.hidden_size))
          score = self.softmax(score)
          pooled_output = torch.sum(score * hidden_states, dim=1)
          pooled_output = self.w_basic(pooled_output)
          pooled_output = self.activation(pooled_output)
          return pooled_output

        elif (self.mode == "Attentive2"): # 2. Attentive Pooling
          score = self.query(hidden_states[:, 1:, :]) # hidden_states[:, 1:, :] = [N, T-1, H] - everything except [CLS]
          score /= torch.sqrt(torch.tensor(self.hidden_size))
          score = self.softmax(score)
          pooled_output = torch.sum(score * hidden_states[:, 1:, :], dim=1) # [N, H]
          pooled_output = torch.cat((hidden_states[:, 0], pooled_output), dim=1) # [N, 2H]
          pooled_output = self.w_2h(pooled_output)
          pooled_output = self.activation(pooled_output)
          return pooled_output

        elif (self.mode == "Min"): # 3-1. Min
          norm = hidden_states.norm(dim=2) # [N, T]
          min_id = torch.argmin(norm, dim=1) # [N]
          min_id = torch.reshape(min_id, (-1, 1, 1)) # [N, 1, 1]
          indices = torch.broadcast_to(min_id, (-1, 1, self.hidden_size)) # [N, 1, H]
          minT = torch.gather(hidden_states, 1, indices) # [N, 1, H]
          pooled_output = self.w_basic(minT.squeeze()) # [N, H]
          pooled_output = self.activation(pooled_output)
          return pooled_output

        elif (self.mode == "Max"): # 3-2. Max
          norm = hidden_states.norm(dim=2) # [N, T]
          max_id = torch.argmax(norm, dim=1) # [N]
          max_id = torch.reshape(max_id, (-1, 1, 1)) # [N, 1, 1]
          indices = torch.broadcast_to(max_id, (-1, 1, self.hidden_size)) # [N, 1, H]
          maxT = torch.gather(hidden_states, 1, indices) # [N, 1, H]
          pooled_output = self.w_basic(maxT.squeeze()) # [N, H]
          pooled_output = self.activation(pooled_output)
          return pooled_output

        elif (self.mode == "Mean"): # 3-3. Mean
          pooled_output = torch.mean(output, dim=1) # [N, H]
          pooled_output = self.w_basic(pooled_output) # [N, H]
          pooled_output = self.activation(pooled_output)
          return pooled_output

        # return pooled_output
        # raise NotImplementedError


class MyBertConfig(BertConfig):
    def __init__(self, pooling_layer_type="CLS", **kwargs):
        super().__init__(**kwargs)
        self.pooling_layer_type = pooling_layer_type


class MyBertModel(BertModel):

    def __init__(self, config: MyBertConfig):
        super(BertModel, self).__init__(config)
        self.config = config

        self.embeddings = BertEmbeddings(config)
        self.encoder = BertEncoder(config)

        if config.pooling_layer_type == "CLS":
            # See src/transformers/models/bert/modeling_bert.py#L869
            self.pooler = BertPooler(config)
        elif config.pooling_layer_type == "MEAN_MAX":
            self.pooler = MeanMaxTokensBertPooler(config)
        elif config.pooling_layer_type == "MINE":
            self.pooler = MyBertPooler(config)
        else:
            raise ValueError(f"Wrong pooling_layer_type: {config.pooling_layer_type}")

        self.init_weights()

    @property
    def pooling_layer_type(self):
        return self.config.pooling_layer_type


class MyBertForSequenceClassification(BertForSequenceClassification):
    def __init__(self, config):
        super(BertForSequenceClassification, self).__init__(config)
        self.num_labels = config.num_labels

        self.bert = MyBertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):
        if self.bert.pooling_layer_type in ["CLS", "MEAN_MAX", "MINE"]:
            return super().forward(
                input_ids, attention_mask, token_type_ids, position_ids, head_mask,
                inputs_embeds, labels, output_attentions, output_hidden_states, return_dict
            )
        else:
            raise NotImplementedError


# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).

![Widget inference on a text classification task](https://github.com/huggingface/notebooks/blob/master/examples/images/text_classification.png?raw=1)

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [4]:
task = "rte"
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [5]:
from datasets import load_dataset, load_metric

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [6]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Reusing dataset glue (/home/dhkim/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

To access an actual element, you need to select a split first, then give an index:

In [8]:
dataset["train"][0]

{'label': 1,
 'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'idx': 0}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [9]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(dataset["train"])

,sentence,label,idx
0,It is Ireland that he is going.,unacceptable,6858
1,I expect to double more than my profits.,unacceptable,5609
2,Someone attended every seminar.,acceptable,1052
3,much armchair is needed.,unacceptable,3464
4,Who has seen my snorkel?,acceptable,6204
5,Paul had an affair,acceptable,8312
6,I know a man who hates me.,acceptable,1256
7,Aphrodite quickly may free the animals,unacceptable,8049
8,The DA proved Jones guilty and the Assistant DA will prove Smith.,acceptable,373
9,The president declared Smith as press secretary.,unacceptable,2970


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [11]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [12]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'matthews_correlation': -0.14859644825447751}

Note that `load_metric` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [13]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [14]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [15]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset:

In [16]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Our friends won't buy this analysis, let alone the next one we propose.


We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [17]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [18]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102], [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [19]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/dhkim/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-0d1ecb480fc526fd.arrow
Loading cached processed dataset at /home/dhkim/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3090a5aefb3d3d57.arrow
Loading cached processed dataset at /home/dhkim/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-bbd0c2d3d6c4be43.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `MyBertForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

**Here, you can change the most important argument in this assignment, `pooling_layer_type`.**

In [20]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

pooling_layer_type = "MEAN_MAX"  # CLS, MEAN_MAX, MINE

config = MyBertConfig.from_pretrained(
    pretrained_model_name_or_path=model_checkpoint,
    pooling_layer_type=pooling_layer_type,
)
model = MyBertForSequenceClassification.from_pretrained(model_checkpoint, config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MyBertForSequenceClassification: ['cls.predictions.bias', 'bert.pooler.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing MyBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MyBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MyBertForSequenceCl

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [21]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [22]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [23]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [24]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MyBertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running training *****
  Num examples = 8551
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1605


Step,Training Loss,Validation Loss,Matthews Correlation
500,0.480900,0.427486,0.536451
1000,0.273200,0.444028,0.611877
1500,0.173100,0.689002,0.570305


The following columns in the evaluation set  don't have a corresponding argument in `MyBertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to test-glue/checkpoint-500
Configuration saved in test-glue/checkpoint-500/config.json
Model weights saved in test-glue/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `MyBertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to test-glue/checkpoint-1000
Configuration saved in test-glue/checkpoint-1000/config.json
Model weights saved in test-glue/checkpoint-1000/pytorch_model.bin
tokenizer confi

TrainOutput(global_step=1605, training_loss=0.2990357324713116, metrics={'train_runtime': 93.4061, 'train_samples_per_second': 274.639, 'train_steps_per_second': 17.183, 'total_flos': 276038894482692.0, 'train_loss': 0.2990357324713116, 'epoch': 3.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [25]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `MyBertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16


{'eval_loss': 0.44402793049812317,
 'eval_matthews_correlation': 0.6118771035334829,
 'eval_runtime': 0.6326,
 'eval_samples_per_second': 1648.781,
 'eval_steps_per_second': 104.333,
 'epoch': 3.0}

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).